In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, find_train_doc_path, test_train_split, clean_1D_arrays, norm_unnorm
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
import random

# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 300

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
Xexp = np.array([[1,2],
        [2,3],
        [4,5]])
Y = np.tile(Xexp, (9,1))
print(Y)

[[1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]
 [1 2]
 [2 3]
 [4 5]]


In [38]:
X = np.array([0,1,2,3,4,5,6,7,8,9])
X1,X2 = np.split(X, [3])
print(X1,X2)

X = np.array([[3]])
print(X.shape)
print(type(X.flatten()[0]))

Y = 12
for i in range(12):
    print(i)

[0 1 2] [3 4 5 6 7 8 9]
(1, 1)
<class 'numpy.int64'>
0
1
2
3
4
5
6
7
8
9
10
11


In [39]:
CS = 2.2

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])
if CS == 2.2:
    skip_param_types = 1 #This is what changes for subpoint
    Theta_True = Constants[skip_param_types:skip_param_types+2].flatten()
    param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
                  4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
    exp_d = 2
    n = 15 #Number of experimental data points to use
    bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                       [ 2,  2,   0,  2,  2,  2, 15,  2]])
    LHS = True

else:
    Constants = Theta_True = np.array([1,-1])
    skip_param_types = 0
    param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
    exp_d = 1
    bounds = np.array([[-2,-2],[2,2]])
    LHS = False
    n = 5

print(Constants.shape)
print(Theta_True.shape)

p = 20
d = len(Theta_True)

obj = np.array(["obj"])
emulator = np.array([False])

if emulator == True:
    t = 20*n
else:
    t = 20

# #Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]
Xexp = clean_1D_arrays(Xexp)

m = Xexp.shape[1]
print(Xexp.shape)

theta_mesh = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds)
print(theta_mesh.shape)

all_data_doc = find_train_doc_path(emulator, obj, d, t)
all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 
print(all_data.shape)

train_data, test_data = test_train_split(all_data, sep_fact = 0.2, shuffle_seed=9)

if CS == 1 and emulator == False:
    train_p, test_p = train_data[:,1:-2], test_data[:,1:-2]
else:
    train_p, test_p = train_data[:,1:-1], test_data[:,1:-1]
# print(train_p)
print(train_p.shape)
print(test_p.shape)

(6, 4)
(8,)
(15, 2)
(400, 8)
(20, 10)
torch.Size([4, 8])
torch.Size([16, 8])


In [45]:
def normalize_x(x_vals, m, Xexp, emulator, norm = True, scaler = None):
    """
    Normalizes or unnormalizes x data from training/testing data and experimental data
    
    Parameters
    ----------
        x_vals: ndarray, x_values to normalize from training/testing data or Xexp when standard approaches are used
        m: int, dimensionality of x data
        Xexp: ndarraay, experimental x values
        emulator: bool, whether GP is emulating fxn or error
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler: None or MinMaxScaler(), used to un-normalize data or normalize data based on another sets normalization
        
    Returns
    -------
        x_scl: ndarray, rescaled values of x
        scaler_x: MinMaxScaler(), scaler used to obtain these values
    """
    x_vals = clean_1D_arrays(x_vals)
    if emulator == True:
        x_scl, scaler_x = norm_unnorm(x_vals, norm, scaler)
#         x_scl, scaler_x = norm_unnorm(x_vals[:,-m:], norm, scaler)
    else:
        x_scl, scaler_x = norm_unnorm(Xexp, norm, scaler)
    return x_scl, scaler_x

def normalize_p_data(param_vals_data, m, emulator, norm = True, scaler = None):
    """
    Normalizes or unnormalizes parameter data from training/testing data
    
    Parameters
    ----------
        param_vals_data: ndarray, parameter values to normalize from training/testing data
        m: int, dimensionality of x data
        emulator: bool, whether GP is emulating fxn or error
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler: None or MinMaxScaler(), used to un-normalize data or normalize data based on another sets normalization
        
    Returns
    -------
        param_data_scl: ndarray, rescaled values of x
        scaler_theta: MinMaxScaler(), scaler used to obtain these values
    """
    if emulator == True:
        if norm == True:
            param_data_scl, scaler_theta = norm_unnorm(param_vals_data[:,0:-m], norm, scaler)
        else:
            param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm, scaler)
    else:
        param_data_scl, scaler_theta = norm_unnorm(param_vals_data, norm, scaler)
    return param_data_scl, scaler_theta

def normalize_p_set(p_set, scaler_theta, norm = True):
    """
    Normalizes or unnormalizes parameter data for theta_set
    
    Parameters
    ----------
        param_vals_data: ndarray, parameter values to normalize from training/testing data
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
           
    Returns
    -------
        p_set_scl: ndarray, rescaled values of x
    """
    p_set_scl, scaler_theta = norm_unnorm(p_set, norm, scaler = scaler_theta)
    return p_set_scl

def normalize_p_true(p_true, scaler_theta, norm= True):
    """
    Normalizes or unnormalizes parameter data for theta_true
    
    Parameters
    ----------
        p_true: ndarray, True parameter values
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
           
    Returns
    -------
        theta_true_scl.flatten(): ndarray, rescaled values of theta_true flattened to correct dimensions
    """    
    theta_true_scl, scaler_theta = norm_unnorm(clean_1D_arrays(p_true, param_clean = True), 
                                               norm, scaler_theta)
    return theta_true_scl.flatten()

def normalize_constants(Constants, p_true, scaler_theta, skip_params, CS, norm = True, scaler_C_before = None, 
                        scaler_C_after = None):
    """
    Normalizes or unnormalizes data for constants
    
    Parameters
    ----------
        Constants: ndarray, ndarray, True values of Muller potential constants OR p_true (CS1 only)
        p_true: ndarray, True parameter values
        scaler_theta: None or MinMaxScaler(), used to (un)normalize data based on another sets normalization
        skip_params: int, number of sets of parameters in Constants to skip before reaching the first iterable parameter row
        CS: float, case study label 
        norm: bool, whether the value will be normalized to 0 and 1 (True) or from 0 and 1 (False). Default True
        scaler_C_before: None or MinMaxScaler(), used to un-normalize normalized constansts before theta constants
        scaler_C_after: None or MinMaxScaler(), used to un-normalize normalized constansts after theta constants
        
    Returns
    -------
        x_scl: ndarray, rescaled values of x
        scaler_x: MinMaxScaler(), scaler used to obtain these values
    """    
    num_param_types = clean_1D_arrays(Constants).shape[1]
    len_param_type = int(len(p_true)/num_param_types)
    if CS == 1:
        Constants_scl = normalize_p_true(p_true, scaler_theta, norm)
    else:
        Constants_before, Constants_theta, Constants_after = np.split(Constants, [skip_params,len_param_type+1])
        print(Constants_before.shape, Constants_theta.shape, Constants_after.shape)
        Constants_before_scl, scaler_C_before = norm_unnorm(Constants_before.T, norm, scaler_C_before)
        
        Constants_theta_scl, scaler_theta =  norm_unnorm(clean_1D_arrays(Constants_theta.flatten(), param_clean = True),
                                                 norm, scaler_theta)
        Constants_theta_scl = Constants_theta_scl.reshape((2,4))
        
        Constants_after_scl, scaler_C_after = norm_unnorm(Constants_after.T, norm, scaler_C_after)
        
        Constants_scl = np.vstack((Constants_before_scl.T, Constants_theta_scl, Constants_after_scl.T))
    return Constants_scl, scaler_C_before, scaler_C_after      

In [46]:
norm = True
X_data_scl, scaler_x = normalize_x(train_p[:,-m:], m, Xexp, emulator, norm)
X_exp_scl = normalize_x(Xexp, m, Xexp, emulator, norm, scaler_x)[0]
# print(X_exp_scl)
train_p_scl, scaler_theta = normalize_p_data(train_p, m, emulator, norm)
# p_set_scl = normalize_p_set(theta_mesh, scaler_theta, norm)
p_true_scl = normalize_p_true(Theta_True, scaler_theta, norm)
# print(p_true_scl)
consatnts_scl, scaler_C_before, scaler_C_after  = normalize_constants(Constants, Theta_True, scaler_theta, skip_param_types, 
                                                                      CS, norm)
print(consatnts_scl)
norm = False
# #Note these 3 lines are useless for emulator = False
# X_data = normalize_x(X_data_scl, m, X_data_scl, emulator, norm, scaler_x)[0]
# print(X_data[3])
# print(train_p[3,-m:])

# x_exp = normalize_x(X_exp_scl, m, X_exp_scl, emulator, norm, scaler_x)[0] #Wrong. Need to fix
# print(x_exp[3])
# print(Xexp[3])

# train_p_rescl = normalize_p_data(train_p_scl, m, emulator, norm, scaler_theta)[0]
# print(train_p_rescl[3])
# # print(train_p[3])
# print(train_p[3,0:-m])

# p_set = normalize_p_set(p_set_scl, scaler_theta, norm)
# print(p_set[3])
# print(theta_mesh[3])

# p_true = normalize_p_true(p_true_scl, scaler_theta, norm)
# print(p_true)
# print(Theta_True)

Constants2 = normalize_constants(consatnts_scl, p_true_scl, scaler_theta, skip_param_types, CS, norm, scaler_C_before, 
                                 scaler_C_after)[0]
print(Constants2)
print(Constants)

(1, 4) (2, 4) (3, 4)
[[ 0.          0.46511628  0.13953488  1.        ]
 [ 0.1068507   0.09384224  0.39060608 -0.05254495]
 [ 0.08827858  0.750685    0.80483466  0.68448759]
 [ 0.          0.          0.3271028   1.        ]
 [ 1.          0.5         0.25        0.        ]
 [ 0.          0.33333333  1.          0.66666667]]
(1, 4) (2, 4) (3, 4)
[[-200.  -100.  -170.    15. ]
 [  -1.    -1.    -6.5    0.7]
 [   0.     0.    11.     0.6]
 [ -10.   -10.    -6.5    0.7]
 [   1.     0.    -0.5   -1. ]
 [   0.     0.5    1.5    1. ]]
[[-200.  -100.  -170.    15. ]
 [  -1.    -1.    -6.5    0.7]
 [   0.     0.    11.     0.6]
 [ -10.   -10.    -6.5    0.7]
 [   1.     0.    -0.5   -1. ]
 [   0.     0.5    1.5    1. ]]


In [ ]:
# print(norm_unnorm(Constants_1_scl.T, norm = False, scaler = scaler_C1))
# print(Constants_1)
m = 2
CS = 2.2

p_vals_scl, scaler_theta = normalize_p_data(train_p, m, emulator, norm = True, scaler = None)
# print(p_vals_scl)

Constants_scl, scaler_C_before, scaler_C_after = normalize_constants(Constants, Theta_True, scaler_theta, skip_params, 
                                                                     CS, norm = True)

constants_rscl, scaler_C_before, scaler_C_after = normalize_constants(Constants_scl, Theta_True, scaler_theta, skip_params, 
                                                                     CS, norm = False, scaler_C_before = scaler_C_before, 
                                                                     scaler_C_after = scaler_C_after)
# constants = normalize_constants(Constants, Theta_True, scaler_theta, skip_params, CS, norm = False)
print(Constants)
print(constants_rscl)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
random.seed(10)
X = np.random.randint(-10,10, size = (3,3))
Y = np.random.randint(-10,10, size = (3,3))
print(X)
# print(Y)
# from sklearn.model_selection import LeaveOneOut

# y_true  = []
# y_pred = []

# print(loo.get_n_splits(X))
# for train_index, test_index in loo.split(X):
#     df_test = X[test_index]
#     df_train = X[train_index]
#     print(df_train, df_test)
    

# scaler =  MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)
# print(X)
# X = scaler.inverse_transform(X)
# print(X)

from bo_functions_generic import norm_unnorm
norm = True
X_scl, scaler_X = norm_unnorm(X, norm = norm)
Y_scl, scaler_Y = norm_unnorm(Y, norm = norm)

print(X_scl, scaler_X)
# print(Y, scaler_Y)

X2 = np.array([[0, 1, 0.33],
               [0.25 , 0.75, 0.66]])

X3 = np.array([[2, 5, -2],
               [-5 , 4, 0]])
print(X3)
X3, scaler_X = norm_unnorm(X3, norm = True, scaler = scaler_X)
print(X3)

In [ ]:
Xexp_bounds = np.array([[-1.5, -0.5],
                        [1 , 2]])
LHS = gen_theta_set(LHS = False, n_points = 20, dimensions = 2, bounds = Xexp_bounds)
# print(LHS)
# print(LHS.reshape((20,20,-1)).T)


X1 = np.linspace(-1.5,1,20)
X2 = np.linspace(-0.5,2,20)
X_mesh = np.meshgrid(X1,X2)
print(np.array(X_mesh))

In [ ]:
# https://realpython.com/python-timer/
import time
start = 3678393
res = datetime.timedelta(seconds =start)
print(res)

time_tot = 0
x = 0
for i in range(89900):
    start2 = time.time()
    x += 1+2
    end = time.time()

    time_tot+= (end - start2)


print(time_tot)

In [ ]:
A = np.array([[4,3,0,0],
              [4,0,0,3]])
print(A[np.nonzero(A)])

In [ ]:
B = np.zeros((3,3))
B = pd.DataFrame(B)
type(B)
isinstance(B, pd.DataFrame)

In [ ]:
np.random.seed(9)
Array_1 = np.random.rand(3,2)
Array_2 = np.random.rand(3,2)

print(Array_1)
print(Array_2)

ds = gdal.Open('image.tif')
# loop through each band

path = 

with open("converted.raw", "wb") as outfile:
    for bi in range(ds.RasterCount):
        band = ds.GetRasterBand(bi + 1)
        # Read this band into a 2D NumPy array
        ar = band.ReadAsArray()
        print('Band %d has type %s'%(bi + 1, ar.dtype))   
        np.save(outfile, ar.astype('uint16'))